In [9]:
# import packages
# install Google Tesseract OCR and change the tesseract_cmd variable
# see more: https://pypi.org/project/pytesseract/

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract as tess
import glob

# If you don't have tesseract executable in your PATH, include the following:
# pytesseract.pytesseract.tesseract_cmd = r'<full_path_to_your_tesseract_executable>'

tess.pytesseract.tesseract_cmd = r"C:\Users\adria\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

In [4]:
# Simple image to string (test)
img = Image.open('D:/Video_Keyframes/keyframes/00032/shot00032_1_RKF.png')
text = tess.image_to_string(img)

print(text)

Creating
the CDSC Hydroponic Farm

 



In [6]:
# check the list of available languages
print(tess.get_languages(config=''))

['deu', 'eng', 'osd']


In [13]:
image_list = []

for filename in glob.glob('D:/Video_Keyframes/keyframes/00032/*.png'):
    img = Image.open(filename)
    image_list.append(tess.image_to_string(img))
    
print(image_list)

[' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', 'jal\n\x0c', ' \n\x0c', ' \n\x0c', 'Creating\nthe CDSC Hydroponic Farm\n\n \n\x0c', ' \n\x0c', 'a\n\n=~\n\nKir\n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', '. a\nI uf\n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', 'I a\napr\nalg\n\x0c', ' \n\x0c', 'Special Thanks\nLee Mandell\nMireille Massac\nDayo Adebayo\nRobert Kwiatkowski\nShirley Paulino\nTimothy Pracher-Dix\nCadet Raphael\nLuis Vargas\n\x0c', 'This program was created using\nthe equipment and facilities of\nBrooklyn Community\nAccess Television.\n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', '\x0c', ' \n\x0c']


In [ ]:
#for folder in glob.glob('D:/Video_Keyframes/keyframes')